<img src='../../media/common/LogoWekeo_Copernicus_RGB_0.png' align='left' height='96px'></img>

<hr>

# Analyzing Wet Snow Extent from SWS Time Series Based on Altitude and Local Aspect

This document includes scripts designed for the analysis of downloaded and extracted data on the local disk of a user.

This script is specifically developed to analyze the extent of wet snow from SWS time series data, considering factors such as altitude and local aspect. By using this script, users can gain valuable insights into the spatial and temporal patterns of wet snow distribution, which are crucial for various environmental and climatic studies.

## Environment Setup
Before we begin, we need to prepare our environment by installing and importing the necessary R packages.

### Load Required Libraries

In [ ]:
library(terra)
library(fs)
library(ggplot2)
library(viridis)
library(pracma)

library(magrittr)
library(dplyr)


## Functions

#### Function to Open and Read GeoTIFF Files Assuming Extracted Products from the Catalog

In [2]:
read_tif <- function(file_tif) {
  ds <- rast(file_tif)

  print(dim(ds))
  
  proj <- crs(ds)
  width <- ncol(ds)
  height <- nrow(ds)
  count <- nlyr(ds)
  meta <- list(
    min = minmax(ds)[1, ],
    max = minmax(ds)[2, ],
    mean = global(ds, fun = mean, na.rm = TRUE),
    sd = global(ds, fun = sd, na.rm = TRUE)
  )
  
  list(data = ds, width = width, height = height, count = count, meta = meta, proj = proj)
}

#### Function to Calculate Aspect from a Digital Elevation Model

In [3]:
aspect <- function(dem) {
  z <- matrix(values(dem), nrow = nrow(dem), ncol = ncol(dem))

  grad <- gradient(z)
  x <- grad$X
  y <- grad$Y
  atan2(-x, y)
}


## Usage

<font color='red'>Enter a valid path and file names into the following box. These are mandatory as input for all following steps:</font>
***

#### <font color='#0080FF'>"path_input"</font>: Enter local path on your machine where products are stored. 
<table><tr><td>
Note: If you copy/paste the path from a windows explorer, add an <b>"r"</b> before the string to interpret backslashes as slashes, as shown in the example below. If your path contains already slashes, remove the <b>"r"</b> in the beginning of the string.
</td></tr></table>

#### <font color='#0080FF'> "files_wsm"</font>: Define the wet snow product file names to be loaded. 
<table><tr><td>
Note: In the template, all files starting with <b>'SWS_'</b> and ending with <b>'_WSM.tif'</b> are searched.
</td></tr></table>

#### <font color='#0080FF'> "file_elev"</font>: Define the digital elevation model file name with the same geometry as the SWS products to be used for the analysis.

<table><tr><td>
Note: In this template, the DEM file name must include <b>"DEM_60m"</b> and extension should be <b>".tif"</b>, and must be stored in the same directory as the SWS products.
</td></tr></table>

In [4]:
path_input <- normalizePath("../../data/download/snow-and-ice/SWS/")

files_wsm <- list.files(path = path_input, pattern = "SWS_.*_WSM\\.tif$", full.names = TRUE)
file_elev <- list.files(path = path_input, pattern = ".*DEM_60m.*\\.tif$", full.names = TRUE)

[1] "/home/vagrant/workspace/wekeo4land/data/download/snow-and-ice/SWS/SWS_20240718T183514_S1A_T27WWN_V101_1_QC_DEM_60m_WSM.tif"
[2] "/home/vagrant/workspace/wekeo4land/data/download/snow-and-ice/SWS/SWS_20240718T183514_S1A_T27WWN_V101_1_WSM.tif"           


#### Open and Read the DEM File Using the Function Defined in Box [2]:

In [ ]:
elev_data <- read_tif(file_elev[1])$data
elev_data[is.na(elev_data)] <- 255

#### Calculate the aspect from the DEM using the function defined in Box [3], and plot the resulting layer.
<table><tr><td>
Note: aspect calculation in the function provides the aspect in radians. For the illustration with the windrose toolbox, aspect information must be converted to degress, scaled between 0 and 360°. So, an offset of 180° is added to the conversion formula to rescale values from range [-180, 180] to [0, 360].
</td></tr></table>

In [ ]:
# Calculate aspect
asp_data <- aspect(elev_data)

# Convert aspect to degrees
aspdeg <- (180 / pi) * asp_data + 180

# Plot the aspect data
plot(aspdeg, main = "Aspect (degrees)")

#### Open and read all Wet Snow products for the Mountains using the function defined in Box [2]:

In [ ]:
wsm_datas <- list()

for (file_wsm in files_wsm) {
  wsm_datas <- append(wsm_datas, list(read_tif(file_wsm)$data))
}

#### Extract the aspect and elevation information for wet snow pixels and add this data to a pandas DataFrame for each SWS product in the time series:
<table><tr><td>
Note: You can activate <b>maxelev</b>, <b>minelev</b>, and <b>elevrange</b> if needed to obtain the minimum and maximum elevation, as well as the elevation range of the selected scene.
</td></tr></table>

In [ ]:
# Function to create and display wind rose plot for each data frame
create_windrose_plot <- function(df) {

  # Create bins for elevation
  df <- df %>%
    mutate(elevation_bin = cut(elevation, breaks = seq(0, 4500, by = 500), include.lowest = TRUE))

  # Create a summary data frame for plotting
  df_summary <- df %>%
    group_by(aspect = cut(aspect, breaks = seq(0, 360, by = 40)), elevation_bin) %>%
    summarize(count = n(), .groups = 'drop')
  
  ggplot(df_summary, aes(x = aspect, y = count, fill = elevation_bin)) +
    geom_col(position = "dodge") +
    coord_polar(start = 0) +
    scale_x_discrete(drop = FALSE) +
    scale_fill_viridis_d() +
    theme_minimal() +
    labs(title = "Wind Rose Plot", x = "Aspect (degrees)", y = "Count") +
    theme(
      axis.title.x = element_blank(),
      axis.title.y = element_blank(),
      panel.grid.major = element_line(color = "grey80"),
      panel.grid.minor = element_blank(),
      panel.background = element_rect(fill = "white")
    )
}

val_wet_snow <- 110
dataframe_list <- list()

for (i in 1:length(wsm_datas)) {
    wsm_data <- wsm_datas[[i]]
    sel <- (values(wsm_data) == val_wet_snow)
    sel[is.na(sel)] <- FALSE

    elev_sel <- elev_data[sel]
    asp_sel <- aspdeg[sel]

    df <- data.frame(elevation = elev_sel, aspect = asp_sel)
    colnames(df) <- c("elevation", "aspect")
    dataframe_list[[i]] <- df
}

print(length(dataframe_list))

# Plot using ggplot2
for (df in dataframe_list) {
  print(length(df))
  if (length(df$elevation) > 0) {
   print(create_windrose_plot(df))
  }
}


***
><span style = "font-family:Verdana; font-size:0.7em">Copyright © <font color='darkblue'>2022</font>, by ENVEO IT GmbH.</span>  
<span style = "font-family:Verdana; font-size:0.7em">Contributors: Gabriele Schwaizer, Lars Keuris</span>  
<span style = "font-family:Verdana; font-size:0.7em">URL: www.enveo.at</span> 
***
<p style = "font-family:Verdana; font-size:0.7em; line-height:0.5">Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files, to use the Software without restriction.</p>  
<p style = "font-family:Verdana; font-size:0.7em; line-height:1.15; text-align:justify">THE SOFTWARE IS PROVIDED AS IS, WITHOUT WARRANTY OF ANYKIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. <b>IN NO EVENT SHALL THE COPYRIGHT HOLDER BE LIABLE FOR ANY CLAIM, DIRECT INDIRECT, SPECIAL, INCIDENTAL, OR CONSEQUENTIAL DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</b></p>

*** 